In [1]:
spam_text= ['Send us your password', 'review us - google', 'Send your password', 'Send us your account password']

ham_text= ['Send us your review', 'google review your password', 'Google needs you to reset your password']

spam = {}
for i in spam_text:
    for j in i.lower().split(' '):
        if j not in spam:
            spam[j] = 1
        else:
            spam[j] += 1

print("Spam Dictionary:")            
print(spam)
print("\n")

ham = {}
for i in ham_text:
    for j in i.lower().split(' '):
        if j not in ham:
            ham[j] = 1
        else:
            ham[j] += 1

print("Ham Dictionary:")
print(ham)


Spam Dictionary:
{'send': 3, 'us': 3, 'your': 3, 'password': 3, 'review': 1, '-': 1, 'google': 1, 'account': 1}


Ham Dictionary:
{'send': 1, 'us': 1, 'your': 3, 'review': 2, 'google': 2, 'password': 2, 'needs': 1, 'you': 1, 'to': 1, 'reset': 1}


In [2]:
def given_is_ham(word, is_ham):
    if is_ham == True:
        thing = ham.get(word)
        if thing is None:
            thing = 0
        return float(thing)/ sum(ham.values())
    if is_ham == False:
        thing = spam.get(word)
        if thing is None:
            thing = 0
        return float(thing)/ sum(spam.values())

In [3]:
given_prob('password', False)

NameError: name 'given_prob' is not defined

In [5]:
def given_there_is_word(word, ham):
    if ham:
        return given_is_ham(word, True)*(float(2)/6) / (given_is_ham('password', False)*(float(len(spam_text))/len(spam_text+ham_text)) + given_is_ham('password', True)*(float(len(ham_text))/len(spam_text+ham_text)))
    else:
        return given_is_ham(word, False)*(float(4)/6) / (given_is_ham('password', False)*(float(len(spam_text))/len(spam_text+ham_text)) + given_is_ham('password', True)*(float(len(ham_text))/len(spam_text+ham_text)))
    

In [6]:
given_there_is_word('password', False)

0.7608695652173914

In [7]:
import numpy as np

In [8]:
def classify_sentence(sentence):
    p_ham = 0
    p_spam = 0
    for word in sentence.lower().split(' '):
        p_ham += np.log(given_there_is_word(word, True)) + np.log(float(2)/6)
        p_spam += np.log(given_there_is_word(word, False)) + np.log(float(4)/6)
    if p_ham > p_spam:
        print('is ham', 1-p_ham/p_spam)
    else:
        print('is spam', 1-p_spam/p_ham)

In [9]:
classify_sentence('Send your password google google google google google google')

is spam 0.42116529444513484


In [12]:
import os
import re
import string
import math
import pandas as pd

class SpamDetector(object):
    """Implementation of Naive Bayes for binary classification"""

    def clean(self, s):
        translator = str.maketrans("", "", string.punctuation)
        return s.translate(translator)

    #  tokenize our string into words
    def tokenize(self, text):
        text = self.clean(text).lower()
        return re.split("\W+", text)

    # count up how many of each word appears in a list of words.
    def get_word_counts(self, words):
        word_counts = {}
        for word in words:
            word_counts[word] = word_counts.get(word, 0.0) + 1.0
        return word_counts

    def fit(self, X, Y):
        """Fit our classifier
        Arguments:
            X {list} -- list of document contents
            y {list} -- correct labels
        """
        self.num_messages = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()

        # Compute log class priors (the probability that any given message is spam/ham),
        # by counting how many messages are spam/ham, 
        # dividing by the total number of messages, and taking the log.
        n = len(X)
        self.num_messages['spam'] = sum(1 for label in Y if label == 'spam')
        self.num_messages['ham'] = sum(1 for label in Y if label == 'ham')
        self.log_class_priors['spam'] = np.log(self.num_messages['spam'] / n )
        self.log_class_priors['ham'] = np.log(self.num_messages['ham'] / n )
        self.word_counts['spam'] = {}
        self.word_counts['ham'] = {}

        # for each (document, label) pair, tokenize the document into words.
        for x, y in zip(X, Y):
            c = 'spam' if y == 'spam' else 'ham'
            counts = self.get_word_counts(self.tokenize(x))
            # For each word, either add it to the vocabulary for spam/ham, 
            # if it isn’t already there, and update the number of counts. 
            for word, count in counts.items():
                # Add that word to the global vocabulary.
                if word not in self.vocab:
                    self.vocab.add(word)
                if word not in self.word_counts[c]:
                    self.word_counts[c][word] = 0.0

                self.word_counts[c][word] += count

    # function to actually output the class label for new data.
    def predict(self, X):
        result = []
        # Given a document...
        for x in X:
            counts = self.get_word_counts(self.tokenize(x))
            spam_score = 0
            ham_score = 0
            # We iterate through each of the words...
            for word, _ in counts.items():
                if word not in self.vocab: continue
                # ... and compute log p(w_i|Spam), and sum them all up. The same will happen for Ham
                # add Laplace smoothing
                # https://medium.com/syncedreview/applying-multinomial-naive-bayes-to-nlp-problems-a-practical-explanation-4f5271768ebf
                log_w_given_spam = np.log( (self.word_counts['spam'].get(word, 0.0) + 1) / (self.num_messages['spam'] + len(self.vocab)) )
                log_w_given_ham = np.log( (self.word_counts['ham'].get(word, 0.0) + 1) / (self.num_messages['ham'] + len(self.vocab)) )

                spam_score += log_w_given_spam
                ham_score += log_w_given_ham
            
            # Then we add the log class priors...
            spam_score += self.log_class_priors['spam']
            ham_score += self.log_class_priors['ham']

            # ... and check to see which score is bigger for that document.
            # Whichever is larger, that is the predicted label!
            if spam_score > ham_score:
                result.append('spam')
            else:
                result.append('ham')
        return result
        

# TODO: Fill in the below function to make a prediction, 
# your answer should match the final number in the below output (0.9641)
if __name__ == '__main__':
    from sklearn.model_selection import train_test_split

    # import/clean/label your data
    data = pd.read_csv('spam.csv',encoding='latin-1')
    data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
    data = data.rename(columns={"v1":'label', "v2":'text'})
    print(data.head())
    tags = data["label"]
    texts = data["text"]

    # create texts and tags
    X, y = texts, tags
    print(len(X))
    
    # transform text into numerical vectors
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    # instantiate your SpamDetector
    MNB = SpamDetector()
    # fit to model, with the trained part of the dataset
    MNB.fit(X_train.values, y_train.values)
    print(MNB.num_messages)
#     print(MNB.word_counts)
    # make predictions
    pred = MNB.predict(X_test.values)
    true = y_test.values
    # test for accuracy
    accuracy = sum(1 for i in range(len(pred)) if pred[i] == true[i]) / float(len(pred))
    print("{0:.4f}".format(accuracy))

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
5572
{'spam': 567, 'ham': 3612}
0.9641


In [13]:
import os
import re
import string
import math
import pandas as pd
class SpamDetector(object):
    """Implementation of Naive Bayes for binary classification"""
    # clean up our string by removing punctuation
    def clean(self, s):
        translator = str.maketrans("", "", string.punctuation)
        return s.translate(translator)
    #  tokenize our string into words
    def tokenize(self, text):
        text = self.clean(text).lower()
        return re.split("\W+", text)
    # count up how many of each word appears in a list of words.
    def get_word_counts(self, words):
        word_counts = {}
        for word in words:
            word_counts[word] = word_counts.get(word, 0.0) + 1.0
        return word_counts
    def fit(self, X, Y):
        """Fit our classifier
        Arguments:
            X {list} -- list of document contents
            y {list} -- correct labels
        """
        self.num_messages = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()
        # Compute log class priors (the probability that any given message is spam/ham),
        # by counting how many messages are spam/ham, 
        # dividing by the total number of messages, and taking the log.
        n = len(X)
        self.num_messages['spam'] = sum(1 for label in Y if label == 'spam')
        self.num_messages['ham'] = sum(1 for label in Y if label == 'ham')
        self.log_class_priors['spam'] = np.log(self.num_messages['spam'] / n )
        self.log_class_priors['ham'] = np.log(self.num_messages['ham'] / n )
        self.word_counts['spam'] = {}
        self.word_counts['ham'] = {}
        # for each (document, label) pair, tokenize the document into words.
        for x, y in zip(X, Y):
            c = 'spam' if y == 'spam' else 'ham'
            counts = self.get_word_counts(self.tokenize(x))
            # For each word, either add it to the vocabulary for spam/ham, 
            # if it isn't already there, and update the number of counts. 
            for word, count in counts.items():
                # Add that word to the global vocabulary.
                if word not in self.vocab:
                    self.vocab.add(word)
                if word not in self.word_counts[c]:
                    self.word_counts[c][word] = 0.0
                self.word_counts[c][word] += count
    # function to actually output the class label for new data.
    def predict(self, X):
        result = []
        # Given a document...
        for x in X:
            counts = self.get_word_counts(self.tokenize(x))
            spam_score = 0
            ham_score = 0
            # We iterate through each of the words...
            for word, _ in counts.items():
                if word not in self.vocab: continue
                # ... and compute log p(w_i|Spam), and sum them all up. The same will happen for Ham
                # add Laplace smoothing
                # https://medium.com/syncedreview/applying-multinomial-naive-bayes-to-nlp-problems-a-practical-explanation-4f5271768ebf
                log_w_given_spam = np.log( (self.word_counts['spam'].get(word, 0.0) + 1) / (self.num_messages['spam'] + len(self.vocab)) )
                log_w_given_ham = np.log( (self.word_counts['ham'].get(word, 0.0) + 1) / (self.num_messages['ham'] + len(self.vocab)) )
                spam_score += log_w_given_spam
                ham_score += log_w_given_ham
            # Then we add the log class priors...
            spam_score += self.log_class_priors['spam']
            ham_score += self.log_class_priors['ham']
            # ... and check to see which score is bigger for that document.
            # Whichever is larger, that is the predicted label!
            if spam_score > ham_score:
                result.append('spam')
            else:
                result.append('ham')
        return result
# TODO: Fill in the below function to make a prediction, 
# your answer should match the final number in the below output (0.9641)
if __name__ == '__main__':
    sd = SpamDetector()
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=19)
    sd.fit(X_train, y_train)
    predictions = np.array(sd.predict(X_test))
    print(np.mean(predictions == y_test))

0.9511844938980617


In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Prepare the dataset
data = pd.read_csv('spam.csv',encoding='latin-1')
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

# create texts and tags
X, y = texts, tags

# split the data into train vs test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# transform text into numerical vectors
vectorizer = CountVectorizer()
X_train_dtm = vectorizer.fit_transform(X_train)
print(X_train_dtm)

# instantiate Multinomial Naive Bayes model
nb = MultinomialNB()
# fit to model, with the trained part of the dataset
nb.fit(X_train_dtm, y_train)
X_test_dtm = vectorizer.transform(X_test)
# make prediction
y_pred_class = nb.predict(X_test_dtm)
# test accurarcy of prediction
metrics.accuracy_score(y_test, y_pred_class)

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
  (0, 3286)	1
  (0, 4747)	2
  (0, 1896)	1
  (0, 875)	2
  (0, 6599)	2
  (0, 801)	1
  (0, 5258)	1
  (0, 7209)	3
  (0, 1559)	1
  (0, 913)	1
  (0, 6623)	3
  (0, 1050)	1
  (0, 5980)	1
  (0, 3530)	1
  (0, 919)	1
  (0, 802)	1
  (0, 819)	1
  (0, 5712)	1
  (0, 6727)	1
  (0, 2112)	1
  (0, 5065)	2
  (0, 7373)	1
  (0, 4176)	2
  (0, 1535)	2
  (0, 6604)	1
  :	:
  (4176, 4747)	1
  (4176, 3252)	1
  (4176, 3416)	1
  (4176, 2304)	1
  (4176, 6638)	1
  (4176, 4450)	1
  (4176, 7163)	1
  (4176, 4219)	1
  (4176, 1590)	1
  (4176, 3439)	1
  (4176, 4833)	1
  (4176, 4894)	1
  (4177, 3647)	1
  (4177, 3252)	1
  (4177, 6074)	1
  (4177, 4125)	1
  (4177, 3162)	1
  (4177

0.9856424982053122

In [14]:
import sys
print(sys.version)

3.7.6 (default, Dec 30 2019, 19:38:28) 
[Clang 11.0.0 (clang-1100.0.33.16)]
